<a href="https://colab.research.google.com/github/abhinivesh-s/Semi-Supervised-Text/blob/main/Semi_Supervised_sklearn_v4_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
import torch.utils.data as Data
import pickle

In [ ]:
print(np.__version__)
import sklearn
print(sklearn.__version__)

1.16.2
0.24.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numpy==1.16.2
import numpy as np

     |████████████████████████████████| 17.3MB 331kB/s 
ERROR: xarray 0.18.2 has requirement numpy>=1.17, but you'll have numpy 1.16.2 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.16.2 which is incompatible.
ERROR: pyerfa 2.0.0 has requirement numpy>=1.17, but you'll have numpy 1.16.2 which is incompatible.
ERROR: pyarrow 3.0.0 has requirement numpy>=1.16.6, but you'll have numpy 1.16.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement numpy>=1.18.5, but you'll have numpy 1.16.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: astropy 4.2.1 has requirement numpy>=1.17, but you'll have numpy 1.16.2 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully unin

In [ ]:
!pip install --upgrade scikit-learn  # Do this to use sklearn SelfTrainingClassifier

     |████████████████████████████████| 22.3MB 1.2MB/s 
ERROR: kapre 0.3.5 has requirement numpy>=1.18.5, but you'll have numpy 1.16.2 which is incompatible.
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
%cd "/content/drive/My Drive/Colab Notebooks/MixText-master"
!ls

/content/drive/My Drive/Colab Notebooks/MixText-master
 code	     LICENSE			'Running MixText v5 ISIS Data.ipynb'
 data	     README.md			'Semi-Supervised sklearn v2.ipynb'
 data_temp  'Running MixText v3.ipynb'	 word2vec_model.bin


In [ ]:
import os

import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

data = fetch_20newsgroups(subset='train', categories=None)
print("%d documents" % len(data.filenames))
print("%d categories" % len(data.target_names))
print()


def eval_and_print_metrics_df(clf, X_train, y_train, X_test, y_test, thresh = None, kbest = None):

    dict1 = {}


    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    
    dict1['Labeled'] = len(X_train) - sum(1 for x in y_train if x == -1)
    dict1['UnLabeled'] = sum(1 for x in y_train if x == -1)


    
    # if sum(1 for x in y_train if x == -1) == 0:
    #     dict1['type'] = 'Supervised'
    # else:
    #     dict1['type'] = 'Semi-Supervised'

    dict1['Threshold'] = thresh
    dict1['K-Best'] = kbest

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))

    dict1['Accuracy'] = accuracy_score(y_test, y_pred)

    print("-" * 10)
    print()

    return dict1




11314 documents
20 categories



In [ ]:
# MLP for NG
from sklearn.neural_network import MLPClassifier

# Parameters
mlp_params = dict(hidden_layer_sizes=(100,50), max_iter=100,activation = 'relu',solver='adam',random_state=1,learning_rate_init=0.01,
                  learning_rate='adaptive')
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_mlp_ng = pd.DataFrame()

    n_list = [10, 20, 30, 40, 50]
    kbest_list=[4, 5, 6, 7, 8]
    threshold = [0.4, 0.5, 0.6, 0.7, 0.8]

    for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = "+str(n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
      X, y = data.data, data.target
      X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)



      unlabeled_mask = np.random.rand(len(y_train)) < 0.5
      X_u50, y_u50 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_train, y_train, unlabeled_mask) if m)))
      
      y_u50 = np.array([-1 for i in y_u50])

      X_50, y_50 = map(list, zip(*((x, y)
                for x, y, m in zip(X_train, y_train, unlabeled_mask) if ~m)))
      




      percentage = 2*(n/100)
      y_mask = np.random.rand(len(y_50)) < percentage

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_50, y_50, y_mask) if m)))

      print("Supervised MLPClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline



      
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', MLPClassifier(**mlp_params)),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_mlp_ng = df_mlp_ng.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(MLPClassifier(**mlp_params), criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_20+X_u50, np.concatenate((y_20, y_u50)), X_test, y_test, thresh = t, kbest = None)
        df_mlp_ng = df_mlp_ng.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = 10~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised MLPClassifier on 10% of the training data:
Number of training samples: 869
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.650
Accuracy Score:  0.6496995404736656
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 5176
Unlabeled samples in training set: 4307
End of iteration 1, added 3708 new labels.
End of iteration 2, added 497 new labels.
End of iteration 3, added 65 new labels.
End of iteration 4, added 33 new labels.
End of iteration 5, added 4 new labels.
Micro-averaged F1 score on test set: 0.662
Accuracy Score:  0.6620714033227288
----------

---------------------------------Threshold =  0.5 ---------------------------------
Number of training samples: 5176
Unlabeled samples in training se

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:619: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Micro-averaged F1 score on test set: 0.818
Accuracy Score:  0.8179568752209261
----------

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = 30~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised MLPClassifier on 30% of the training data:
Number of training samples: 2541
Unlabeled samples in training set: 0


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:619: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Micro-averaged F1 score on test set: 0.784
Accuracy Score:  0.7836691410392365
----------

SelfTrainingClassifier on 30% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 6848
Unlabeled samples in training set: 4307


KeyboardInterrupt: ignored

In [ ]:
df_mlp_ng.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']

Threshold               NaN       0.4       0.5       0.6       0.7       0.8
Labeled UnLabeled                                                            
869.0   0.0        0.649700       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.662071  0.688583  0.703075  0.716154  0.707671
1695.0  0.0        0.762460       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.767409  0.811948  0.806645  0.751502  0.817957
2541.0  0.0        0.783669       NaN       NaN       NaN       NaN       NaN

In [ ]:
%cd "/content/drive/My Drive/Colab Notebooks/MixText-master"
!ls

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
def train_val_split(labels, n_labeled_per_class, unlabeled_per_class, n_labels, seed=0):
    """Split the original training set into labeled training set, unlabeled training set, development set

    Arguments:
        labels {list} -- List of labeles for original training set
        n_labeled_per_class {int} -- Number of labeled data per class
        unlabeled_per_class {int} -- Number of unlabeled data per class
        n_labels {int} -- The number of classes

    Keyword Arguments:
        seed {int} -- [random seed of np.shuffle] (default: {0})

    Returns:
        [list] -- idx for labeled training set, unlabeled training set, development set
    """
    np.random.seed(seed)
    labels = np.array(labels)
    train_labeled_idxs = []
    train_unlabeled_idxs = []
    val_idxs = []

    for i in range(n_labels):
        idxs = np.where(labels == i)[0]
        np.random.shuffle(idxs)
        if n_labels == 2:
            # IMDB
            
            
            
            n_unlabeled_per_class = unlabeled_per_class   #10, 100, 500, 1000, 2500
            train_pool = np.concatenate((idxs[:500], idxs[5500:-2000]))
            train_labeled_idxs.extend(train_pool[:n_labeled_per_class])
            train_unlabeled_idxs.extend(idxs[500: 500 + n_unlabeled_per_class])
            val_idxs.extend(idxs[-2000:])
        
        
        
            # train_pool = np.concatenate((idxs[:500], idxs[5500:-2000]))
            # train_labeled_idxs.extend(train_pool[:n_labeled_per_class])
            # train_unlabeled_idxs.extend(
            #     idxs[500: 500 + 5000])
            # val_idxs.extend(idxs[-2000:])
            

    return train_labeled_idxs, train_unlabeled_idxs, val_idxs



import re
#Removes Punctuations
def remove_punctuations(data):
    punct_tag=re.compile(r'[^\w\s]')
    data=punct_tag.sub(r'',data)
    return data

#Removes HTML syntaxes
def remove_html(data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data

#Removes URL data
def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data



def clean(train_df):
    
    train_df['review']=train_df['review'].apply(lambda z: remove_punctuations(z))
    train_df['review']=train_df['review'].apply(lambda z: remove_html(z))
    train_df['review']=train_df['review'].apply(lambda z: remove_url(z))
    # train_df['review']=train_df['review'].apply(lambda z: remove_emoji(z))
    
    train_df['review']=train_df['review'].apply(lambda z: word_tokenize(z))
    
    # lemmatizer = WordNetLemmatizer()
    # train_df['review']=train_df['review'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    train_df['review']=train_df['review'].apply(lambda x: ' '.join(x))
    
    return train_df

In [ ]:
import os

import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report



# Parameters
sdg_params = dict(alpha=1e-5, penalty='l2', loss='log', random_state=0)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)

# Supervised Pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(**sdg_params)),
])

# # Supervised Pipeline
# pipeline = Pipeline([
#     ('vect', CountVectorizer(**vectorizer_params)),
#     ('tfidf', TfidfTransformer()),
#     ('clf', SVC(probability=True, gamma="auto")),
# ])

# SelfTraining Pipeline
st_pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    ('clf', SelfTrainingClassifier(SGDClassifier(**sdg_params), verbose=True)),
])

# # SelfTraining Pipeline
# st_pipeline = Pipeline([
#     ('vect', CountVectorizer(**vectorizer_params)),
#     ('tfidf', TfidfTransformer()),
#     ('clf', SelfTrainingClassifier(SVC(probability=True, gamma="auto") )),
# ])





# LabelSpreading Pipeline
ls_pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    # LabelSpreading does not support dense matrices
    ('todense', FunctionTransformer(lambda x: x.todense())),
    ('clf', LabelSpreading()),
])


def eval_and_print_metrics(clf, X_train, y_train, X_test, y_test):
    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # if -1 in y_train:
    #   print("Y-PRED-PROBA", clf.predict_proba(X_train))

    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))
    print("Classification Report: \n", classification_report(y_test, y_pred))
    print("-" * 10)
    print()



def eval_and_print_metrics_df(clf, X_train, y_train, X_test, y_test, thresh = None, kbest = None):

    dict1 = {}


    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    
    dict1['Labeled'] = len(X_train) - sum(1 for x in y_train if x == -1)
    dict1['UnLabeled'] = sum(1 for x in y_train if x == -1)


    
    # if sum(1 for x in y_train if x == -1) == 0:
    #     dict1['type'] = 'Supervised'
    # else:
    #     dict1['type'] = 'Semi-Supervised'

    dict1['Threshold'] = thresh
    dict1['K-Best'] = kbest

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))

    dict1['Accuracy'] = accuracy_score(y_test, y_pred)

    print("-" * 10)
    print()

    return dict1

In [ ]:
# MLP for IMDB
from sklearn.ensemble import RandomForestClassifier

# Parameters
# mnb_params = 
mlp_params = dict(hidden_layer_sizes=(100,50), max_iter=50,activation = 'relu',solver='adam',random_state=1)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_mlp_imdb_2k = pd.DataFrame()

    n_list = [10, 50, 200, 500, 1000]
    kbest_list=[4, 5, 6, 7, 8]
    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]

for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 1000

      data_path = './data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised MLPClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', MLPClassifier(**mlp_params)),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_mlp_imdb_2k = df_mlp_imdb_2k.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(MLPClassifier(**mlp_params), criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_train, y_train, X_test, y_test, thresh = t, kbest = None)
        df_mlp_imdb_2k = df_mlp_imdb_2k.append(temp, ignore_index=True)

    


In [ ]:
df_mlp_imdb_2k.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']